In [2]:
import collections
import logging
import os
import pathlib
import re
import string
import sys
import time

import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow as tf

In [3]:
logging.getLogger('tensorflow').setLevel(logging.ERROR)  # suppress warnings


In [4]:
examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en', with_info=True,
                               as_supervised=True)
train_examples, val_examples = examples['train'], examples['validation']

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

...

Generating validation examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\Owner\tensorflow_datasets\ted_hrlr_translate\pt_to_en\incomplete.ET38MM_1.0.0\ted_hrlr_tran…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\Owner\tensorflow_datasets\ted_hrlr_translate\pt_to_en\incomplete.ET38MM_1.0.0\ted_hrlr_tran…

Dataset ted_hrlr_translate downloaded and prepared to C:\Users\Owner\tensorflow_datasets\ted_hrlr_translate\pt_to_en\1.0.0. Subsequent calls will reuse this data.


In [5]:
for pt_examples, en_examples in train_examples.batch(3).take(1):
  for pt in pt_examples.numpy():
    print(pt.decode('utf-8'))

  print()

  for en in en_examples.numpy():
    print(en.decode('utf-8'))

e quando melhoramos a procura , tiramos a única vantagem da impressão , que é a serendipidade .
mas e se estes fatores fossem ativos ?
mas eles não tinham a curiosidade de me testar .

and when you improve searchability , you actually take away the one advantage of print , which is serendipity .
but what if it were active ?
but they did n't test for curiosity .


In [6]:
model_name = "ted_hrlr_translate_pt_en_converter"
tf.keras.utils.get_file(
    f"{model_name}.zip",
    f"https://storage.googleapis.com/download.tensorflow.org/models/{model_name}.zip",
    cache_dir='.', cache_subdir='', extract=True
)

184801/184801 [==============================] - 0s 0us/step


'.\\ted_hrlr_translate_pt_en_converter.zip'

In [7]:
tokenizers = tf.saved_model.load(model_name)


In [8]:
[item for item in dir(tokenizers.en) if not item.startswith('_')]

['detokenize',
 'get_reserved_tokens',
 'get_vocab_path',
 'get_vocab_size',
 'lookup',
 'tokenize',
 'tokenizer',
 'vocab']

In [9]:
for en in en_examples.numpy():
  print(en.decode('utf-8'))

and when you improve searchability , you actually take away the one advantage of print , which is serendipity .
but what if it were active ?
but they did n't test for curiosity .


In [10]:
encoded = tokenizers.en.tokenize(en_examples)

for row in encoded.to_list():
  print(row)

[2, 72, 117, 79, 1259, 1491, 2362, 13, 79, 150, 184, 311, 71, 103, 2308, 74, 2679, 13, 148, 80, 55, 4840, 1434, 2423, 540, 15, 3]
[2, 87, 90, 107, 76, 129, 1852, 30, 3]
[2, 87, 83, 149, 50, 9, 56, 664, 85, 2512, 15, 3]


In [11]:
round_trip = tokenizers.en.detokenize(encoded)
for line in round_trip.numpy():
  print(line.decode('utf-8'))

and when you improve searchability , you actually take away the one advantage of print , which is serendipity .
but what if it were active ?
but they did n ' t test for curiosity .


In [12]:
tokens = tokenizers.en.lookup(encoded)
tokens

<tf.RaggedTensor [[b'[START]', b'and', b'when', b'you', b'improve', b'search', b'##ability',
  b',', b'you', b'actually', b'take', b'away', b'the', b'one', b'advantage',
  b'of', b'print', b',', b'which', b'is', b's', b'##ere', b'##nd', b'##ip',
  b'##ity', b'.', b'[END]']                                                 ,
 [b'[START]', b'but', b'what', b'if', b'it', b'were', b'active', b'?',
  b'[END]']                                                           ,
 [b'[START]', b'but', b'they', b'did', b'n', b"'", b't', b'test', b'for',
  b'curiosity', b'.', b'[END]']                                          ]>

In [13]:
def tokenize_pairs(pt, en):
    pt = tokenizers.pt.tokenize(pt)
    # Convert from ragged to dense, padding with zeros.
    pt = pt.to_tensor()

    en = tokenizers.en.tokenize(en)
    # Convert from ragged to dense, padding with zeros.
    en = en.to_tensor()
    return pt, en

In [14]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64

In [15]:
def make_batches(ds):
  return (
      ds
      .cache()
      .shuffle(BUFFER_SIZE)
      .batch(BATCH_SIZE)
      .map(tokenize_pairs, num_parallel_calls=tf.data.AUTOTUNE)
      .prefetch(tf.data.AUTOTUNE))


train_batches = make_batches(train_examples)
val_batches = make_batches(val_examples)

시계열 연습

간단하게 셀프 어텐션 계산 결과를 보자.

`x` : 3개의 원래 벡터와 그것의 스케일된 복사본 3개가 하나의 6-step 시퀀스로 만들었음

- `x = [v₁, v₂, v₃, 2·v₁, 2·v₂, 2·v₃]`

- 유사한 정보가 반복된 시퀀스를 만들어서, attention이 어떤 식으로 유사성을 인식하는지 확인하려는 목적

`layers.MultiHeadAttention`의 결과는 1행과 4행, 2행과 5행, 3행과 6행이 비슷한 값을 가지게 됨

In [ ]:
temp = np.random.randn(1, 3, 1)
x = np.concatenate([temp, 1.1*temp], axis = 1)

layers.MultiHeadAttention(num_heads=1, key_dim=4)(query=x, value=x)

layers.MultiHeadAttention(num_heads=2, key_dim=4, output_shape = 2*4)(query=x, value=x)

import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import MultiHeadAttention

# 입력 시퀀스: 2개 타임스텝, 각 벡터는 2차원
x = tf.constant([[[1.0, 0.0],   # pos 0
                  [0.0, 1.0]]], # pos 1
                dtype=tf.float32)  # shape: (1, 2, 2)

# MultiHeadAttention 정의 (1-head, dim=2)
mha = MultiHeadAttention(num_heads=1, key_dim=2)

# 예측 + attention score 반환
output, attn_scores = mha(query=x, value=x, return_attention_scores=True)

print("Output:\n", output.numpy()[0])
print("\nAttention weights:\n", attn_scores.numpy()[0][0])


encoder_inputs = keras.Input(shape=(timestep, dim))
pos_encoder_inputs = PositionalEncoding(timestep, dim)(encoder_inputs)
MHA_layer = layers.MultiHeadAttention(key_dim=256, num_heads=2)
x = MHA_layer(pos_encoder_inputs, pos_encoder_inputs)
x = layers.Dense(units=50, activation="relu")(x + encoder_inputs)
x = layers.Dense(units=encoder_inputs.shape[-1])(x)
model = keras.Model(encoder_inputs, x)
model.summary()

# 데이터 생성
num_samples = 10000
input_dim = 4  # 입력 차원
timesteps = 6  # 시퀀스 길이

temp = np.random.uniform(-1, 1, size=(num_samples, timesteps, input_dim)).astype(np.float32)
temp1 = np.random.uniform(-100, 100, size=(num_samples, timesteps - 1, input_dim)).astype(np.float32)
temp2 = np.ones((num_samples, 1, input_dim)).astype(np.float32)

input_sequences = temp #np.concatenate([temp1, temp2], axis=1)
target_sequences = temp.copy()
target_sequences = np.abs(input_sequences)

for i in range(num_samples):
    ri = np.random.randint(1, timesteps)
    input_sequences[i, ri, :] = np.ones((1, 1, input_dim))
    target_sequences[i, ri, :] = target_sequences[i, ri-1, :] 

num_samples = 10000
input_dim = 4  # 입력 차원
timesteps = 6  # 시퀀스 길이

temp = np.random.uniform(-1, 1, size=(num_samples, timesteps, input_dim)).astype(np.float32)
input_sequences = temp #np.concatenate([temp1, temp2], axis=1)

for i in range(2, timesteps):
    input_sequences[:, i, :] = i *np.prod(input_sequences[:, i-2:i, :], axis = 1)


target_sequences = np.zeros_like(input_sequences)
target_sequences[:, :-1, :] = input_sequences[:, 1:, :]
target_sequences[:, -1, :] = np.sum(target_sequences[:, -3:-1, :], axis = 1)

num_samples = 10000
input_dim = 4  # 입력 차원
timesteps = 6  # 시퀀스 길이

temp = np.random.uniform(-1, 1, size=(num_samples, timesteps, input_dim)).astype(np.float32)
input_sequences = temp #np.concatenate([temp1, temp2], axis=1)

target_sequences = input_sequences.copy()

for i in range(num_samples):
    if np.all(target_sequences[i, -1, :] > 0):
        target_sequences[i, -1, :] = target_sequences[i, 0, :]
    if np.all(target_sequences[i, -1, :] < 0):
        target_sequences[i, -1, :] = target_sequences[i, 1, :]

input_sequences[1,:,:]

target_sequences[1,:,:]

model.compile(optimizer='adam', loss='mean_squared_error')
# 모델 훈련
model.fit(
    input_sequences, target_sequences,
    validation_split=0.2,
    epochs=20,
    batch_size=32
)

temp = np.random.uniform(-1, 1, size=(1, timesteps, input_dim)).astype(np.float32)
new_input_sequence = temp #np.concatenate([temp1, temp2], axis=1)

for i in range(2, timesteps):
    input_sequences[:, i, :] = np.sum(input_sequences[:, i-2:i, :], axis = 1)

print(new_input_sequence)
model.predict(new_input_sequence)

MHA_layer(new_input_sequence, new_input_sequence)

new_input_sequence = 0.01 * np.array([[[11., 21., 93., 56.],
                                       [90., -91., -93., 96.],
                                       [21., -41., -93., -55.],
                                       [-79., -22., -34., -1.],
                                       [-79., -2., -37., -1.],
                                       [10., 10., 10., 10.]]])

new_input_sequence[0, -1, :] = np.sum(new_input_sequence[0, :-1, :], axis = 0)
print(new_input_sequence)
model.predict(new_input_sequence)

MHA_layer(new_input_sequence, new_input_sequence)

new_input_sequence = 0.01 * np.array([[[11., 21., 93., 56.],
                                       [90., -91., -93., 96.],
                                       [21., -41., -93., -55.],
                                       [-79., -22., -34., -1.],
                                       [-79., -2., -37., -1.],
                                       [-10., -10., -10., -10.]]])

new_input_sequence[0, -1, :] = np.sum(new_input_sequence[0, :-1, :], axis = 0)
MHA_layer(new_input_sequence, new_input_sequence)